In [39]:
import numpy as np
from numpy import exp
import matplotlib.pyplot as plt
import pandas as pd

In [40]:
df = pd.read_csv("marriage_proposal.csv")

height = df["Height"].to_numpy()
age = df["Age"].to_numpy()
Income = df["Income"].to_numpy()
RomanticGestureScore = df["RomanticGestureScore"].to_numpy()
CommunicationScore   = df["CompatibilityScore"].to_numpy()
DistanceKM  = df["DistanceKM"].to_numpy()
Response  = df["Response"].to_numpy()
AgeCategory  = np.where(df["AgeCategory"].to_numpy() == 'Young', 0, 
                    np.where(df["AgeCategory"].to_numpy() == 'Senior', 1, 
                    np.where(df["AgeCategory"].to_numpy() == 'Middle-aged', 2, -1)))


In [41]:
numEpocas = 100   
q = len(age)               
eta = 0.01            
m =  7               
N = 16    
N2 = 8
N3 = 4
N4 = 2             
L = 1   

In [42]:
W1 = np.random.random((N, m + 1))
W2 = np.random.random((N2, N + 1))
W3 = np.random.random((N3, N2 + 1))
W4 = np.random.random((N4, N3 + 1))
W5 = np.random.random((L, N4 + 1))
bias = 1
X = np.vstack((height,age,Income,RomanticGestureScore,CommunicationScore,DistanceKM,AgeCategory))
E = np.zeros(q)
Etm = np.zeros(numEpocas) 

In [43]:
print(X)

def think_Hyperbolic(W,array):
    return np.tanh(W.dot(array))

def think_Relu(W,array):
      return np.maximum(W.dot(array))

def think_sigmoid(W, array):
        return 1 / (1 + exp(-(W.dot(array))))


[[  157   141   143 ...   172   192   140]
 [   23    80    56 ...    89    60    75]
 [ 9813  9543 43513 ... 37641 15706 47621]
 ...
 [    4     9     2 ...     3     6     9]
 [   81    14    55 ...     6    94    51]
 [    0     1     2 ...     1     1     1]]


In [49]:
for i in range(numEpocas):
    for j in range(len(X[0])):

        Xb = np.hstack((bias,X[:,j]))
        o1 = think_Hyperbolic(W1,Xb)
        o1b = np.insert(o1,0,bias)
        o2 = think_Hyperbolic(W2,o1b)
        o2b = np.insert(o2,0,bias)
        o3 = think_Hyperbolic(W3,o2b)
        o3b = np.insert(o3,0,bias)
        o4 = think_Hyperbolic(W4,o3b)
        o4b = np.insert(o4,0,bias)
        Y = think_sigmoid(W5,o4b)

        e = Response[j] - Y
        E[j] = (e.transpose().dot(e))/2

        delta5 = np.diag(e).dot((1 - Y*Y))
        vdelta5 = (W5.transpose()).dot(delta5)
        print(o4b)
        delta4 = np.diag(1 - o4b*o4b).dot(vdelta5)
        vdelta4 = (W4.transpose()).dot(delta4)
        delta3 = np.diag(1 - o3b*o3b).dot(vdelta4)
        vdelta3 = (W3.transpose()).dot(delta3)
        delta2 = np.diag(1 - o2b*o2b).dot(vdelta2)
        vdelta2 = (W3.transpose()).dot(delta2)
        delta1 = np.diag(1 - o1b*o1b).dot(vdelta1)
        vdelta1 = (W3.transpose()).dot(delta1)

        W1 = W1 + eta*(np.outer(delta1[1:], Xb))
        W2 = W2 + eta*(np.outer(delta2, o1b))
        W3 = W3 + eta*(np.outer(delta3, o2b))
        W4 = W4 + eta*(np.outer(delta4, o3b))
        W5 = W5 + eta*(np.outer(delta5, o4b))

Etm[i] = E.mean()



[1.         0.99612341 0.99460667]


ValueError: shapes (5,2) and (3,) not aligned: 2 (dim 1) != 3 (dim 0)